In [2]:
%env LC_ALL=C
from cgitb import text
import spacy
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
from spacy.tokens import DocBin
from multiprocessing import Process, freeze_support, set_start_method
import os
import json
import pickle

env: LC_ALL=C


/home/davidlin/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_word_vectors(model_name, word_vectors):
    import spacy
    import subprocess
    import sys
    subprocess.run([sys.executable,
                    "-m",
                    "spacy",
                    "init",
                    "vectors",
                    "en",
                    word_vectors,
                    model_name
                        ]
                    )
    print (f"New spaCy model created with word vectors. File: {model_name}")

In [4]:
spacy.require_gpu()
#load_word_vectors("./models/model-best", "trainset/word_vecs.txt")
nlp = spacy.load("./models/model-best")
dataframe = pd.read_csv('./dataset/Listing_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')

In [5]:
listing_titles = [(dataframe["Title"][i], i) for i in range(0, len(dataframe["Title"]))][5000:30000]

In [6]:
docs = list(nlp.pipe(listing_titles, as_tuples=True, batch_size=10000))

In [7]:
output = []

In [8]:
result = []

with tqdm(total = 25000) as pbar:
    buffer = ""
    for doc, i in docs:
        for ent in doc.ents:
            if(ent.label_ != "No Tag" and ent.label_ != "Obscure"):
                result.append(str(i + 1) + '\t' + ent.label_ + '\t' + ent.text)
        pbar.update(1)
        
        


100%|██████████| 25000/25000 [00:00<00:00, 69744.19it/s]


In [9]:
os.makedirs("./output", exist_ok=True)
output = open("output/output.tsv", "w", encoding="utf-8")  # write mode
output.write('\n'.join(result))
output.close()

In [20]:
realdocs = [doc for doc, i in docs]
doc_bin = DocBin()
for doc in realdocs:
    doc_bin.add(doc)

In [22]:
print(realdocs[0])

SALE ! Kamen Rider Gaim Belt Set No. 802


In [11]:
output = open("output/output.spacy", "wb")  # write mode
output.write(doc_bin.to_bytes())
output.close()